<a href="https://colab.research.google.com/github/Vandanarajotiya/Dog_vision/blob/main/Dog_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##!unzip "drive/MyDrive/Dog Vision/dog-breed-identification.zip" -d "drive/MyDrive/Dog Vision/"

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
print("TF version:", tf.__version__)
print("TF Hub version:",hub.__version__)

#check for GPU availability
print("GPU", "available (YESSSSS!!!!!!)" if tf.config.list_physical_devices("GPU") else "not available: (")

In [ ]:
!ls /content/sample_data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
lables_csv = pd.read_csv("drive/MyDrive/Dog Vision/labels.csv")
print(lables_csv.describe())
print(lables_csv.head())


In [ ]:
lables_csv.head()

In [ ]:
lables_csv["breed"].value_counts().plot.bar(figsize=(20,10))

In [ ]:
lables_csv["breed"].value_counts().median()

In [ ]:
# !rm -rf "drive/MyDrive/Dog Vision/train"
# !unzip "drive/MyDrive/Dog Vision/train.zip" -d "drive/MyDrive/Dog Vision/"


In [ ]:
from IPython.display import Image
Image("drive/MyDrive/Dog Vision/train/001513dfcb2ffafc82cccf4d8bbaba97.jpg")

In [ ]:
filename = ["drive/MyDrive/Dog Vision/train/" + fname + ".jpg" for fname in lables_csv["id"]]
filename[:10]

In [ ]:
import os
if len(os.listdir("drive/MyDrive/Dog Vision/train/")) == len(filename):
  print("Filenames match actual amount of files!!! ")
else:
  print("Filename do no match actual amount of filels , check the traget directory.")


In [ ]:
Image(filename[9000])

In [ ]:
import numpy as np
lables = lables_csv["breed"].to_numpy()
lables

In [ ]:
len(lables)

In [ ]:
if len(lables) == len(filename):
  print("number of lables matches number of filenames")
else:
  print("number of lables do not match number of filenames. check data directoris!")

In [ ]:
#find the unique
unique_breeds = np.unique(lables)
len(unique_breeds)

In [ ]:
boolean_lables = [lables == unique_breeds for lables in lables]
boolean_lables[:2]

In [ ]:
NUM_IMAGES = 1000 #@param {type: "slider" , min:1000,max:10000 , step:1000}


In [ ]:
from sklearn.model_selection import train_test_split
X,y= filename , boolean_lables
X_train, X_val ,y_train , y_val = train_test_split(X[:NUM_IMAGES],
                                                   y[:NUM_IMAGES],
                                                   test_size=0.2,
                                                   random_state=42)

len(X_train),len(y_train),len(X_val),len(y_val)

In [ ]:
X_train[:5] , y_train[:2]

In [ ]:
#convert image to numpy array
from matplotlib.pyplot import imread
image = imread(filename[42])
image.shape

In [ ]:
tf.constant(image)[:2]

In [ ]:
IMG_SIZE= 224

def process_image(image_path , img_size = IMG_SIZE):
  """
  Takes an image file path and turns it into a Tensor.
  """
  # Read in an image file
  image = tf.io.read_file(image_path)
  #Turn the jpeg image to numerical Tensor with 3 color channels
  image = tf.io.decode_jpeg(image,channels=3)
  # Convert the colour chennel values from 0-255 to 0-1 values
  image = tf.image.convert_image_dtype(image , tf.float32)
  # Resize the image to our desired value (224 , 224)
  image = tf.image.resize(image,size=[IMG_SIZE,IMG_SIZE])

  return image




In [ ]:
#Tuning our data into batches
def get_image_lable(image_path , lable):
  """
  Taken an image file path name and the associated lable,
  processes the image and returns a lable type of (image, lable)
  """
  image = process_image(image_path)
  return image, lable

In [ ]:
process_image(X[42], y[42])

In [ ]:
BATCH_SIZE = 32
# crete a function to turn data into batches
def create_data_batches(X,y=None,batch_size=BATCH_SIZE,valid_data=False,test_data=False):
  """
  create batches of data out of image (x)and lable (y)
  suffles the data if it's traning data but doesn't shuffle if it's validation data
  also accepts test data as input (no lables)
  """
  if test_data:
    print("Creating test data batches")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X)))
    data_batch = data.map(process_image).batch(BATCH_SIZE)
    return data_batch
# if the data is a valid dataset , we don't need shuffle it
  elif valid_data:
    print("Creating validation data batches....")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X),
                                               tf.constant(y)))
    data_batch = data.map(get_image_lable).batch(BATCH_SIZE)
    return data_batch

  else:
    print("Creating training data batches....")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X),
                                               tf.constant(y)))
    data = data.shuffle(buffer_size=len(X))
    data = data.map(get_image_lable)

    #turn the traning data into batchs
    data_batch = data.batch(BATCH_SIZE)
  return data_batch

In [ ]:
train_data = create_data_batches(X_train,y_train)
valid_data = create_data_batches(X_val,y_val,valid_data=True)

In [ ]:
# Different attributes of out data batches
train_data.element_spec , valid_data.element_spec

Training → shuffle + preprocess + batch
Validation → preprocess + batch (no shuffle)
Test → preprocess + batch (sirf images, no labels)

In [ ]:
import matplotlib.pyplot as plt
# Ceate a function for viewing images in a data batch
def show_25_images(images,lables):
  """
  displays 25 images from a data batch
  """
  plt.figure(figsize=(10,10))
  for i in range(25):
    ax = plt.subplot(5,5,i+1)
    plt.imshow(images[i])
    plt.title(unique_breeds[lables[i].argmax()])
    # grid lines off
    plt.axis("off")


In [ ]:
train_images , train_lables = next(train_data.as_numpy_iterator())
show_25_images(train_images, train_lables)

In [ ]:
val_images , val_lables = next(valid_data.as_numpy_iterator())
show_25_images(val_images,val_lables)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import Sequential, layers

IMG_SIZE = 224
NUM_CLASSES = len(unique_breeds)  

def create_model():
    base_model = MobileNetV2(
        include_top=False,         
        input_shape=(IMG_SIZE, IMG_SIZE, 3),          
        weights='imagenet'         
    )
    base_model.trainable = False 
    x = layers.GlobalAveragePooling2D()(base_model.output)
    output = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    model = tf.keras.Model(inputs=base_model.input,outputs = output)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

model = create_model()


In [ ]:
#Lode tenserboard notebook extention
%load_ext tensorboard

In [ ]:
import datetime

def create_tensorboard_callback():
  logdir = os.path.join("drive/MyDrive/Dog Vision/logs",
                        datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  return tf.keras.callbacks.TensorBoard(logdir)

In [ ]:
#Create Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = "val_accuracy",
                                                  patience = 3)

Training a Data

In [ ]:
NUM_EPOCHS = 100 #@param {type: "slider" , min:10 , max:100 , step:10}

In [ ]:
print("GPU" , "available (YESSSS!!!!!!!!)" if tf.config.list_physical_devices("GPU") else "not available (")

In [ ]:
#Build a function to train and return trained model
def train_model():
  """
  Train aa givin model and return a trained version
  """
  #create a model
  model = create_model()
  #Create a Tensorboard session evertime train a model
  TensorBoard =  create_tensorboard_callback()

  #fit the model to the data passing it the callback we created
  model.fit(x=train_data,
            epochs = NUM_EPOCHS,
            validation_data = valid_data,
            validation_freq = 1,
            callbacks = [TensorBoard , early_stopping])
  return model

In [ ]:
model = train_model()

In [ ]:
valid_data

In [ ]:
#making and Evaluationg predictions using a traind model
prediction = model.predict(valid_data , verbose=1)
prediction

In [ ]:
prediction.shape

In [ ]:
len(y_train)

In [ ]:
len(unique_breeds)

In [ ]:
#First Prediction
index = 0
print(f"max value (probability of predication): {np.max(prediction[index])}")
print(f"Sum :{np.sum(prediction[index])}")
print(f"Max index : {np.argmax(prediction[index])}")
print(f"Prediction :{unique_breeds[np.argmax(prediction[index])]}")


In [ ]:
#Turn prediction probabilities into their respactive lables
def get_pred_lable(prediction_probabilities):
  """
  Turn an array predictions probabilities into lables
  """
  return unique_breeds[np.argmax(prediction_probabilities)]

# get a prediction lable based on an array on prediction probabilities
pred_lable = get_pred_lable(prediction[9])
pred_lable

In [ ]:
valid_data

In [ ]:
#Create a function unbatch abatch dataset
def unbatchify(data):
  """
  Takes a batch dataset of (image.lables) Tensors and returns separate arrays
  of images and lables
  """
  images = []
  lables = []

  for image , lable in data.unbatch().as_numpy_iterator():
    images.append(image)
    lables.append(unique_breeds[np.argmax(lable)])
  return images, lables


val_images , val_lables = unbatchify(valid_data)
val_images[0],val_lables[0]

In [ ]:
get_pred_lable(val_lables[0])

In [ ]:
def plot_pred(prediction_probabilities , lables , images , n=1):
  """
  View the prediction, ground truth and image and semple
  """
  pred_prob, true_lable , image = prediction_probabilities[n] , lables[n],images[n]

  pred_lable = get_pred_lable(pred_prob)
  plt.imshow(image)
  plt.xticks([])
  plt.yticks([])

  plt.title("{} {:2.0f}% {}".format(pred_lable ,
                                      np.max(pred_prob)*100,
                                      true_lable ))


In [ ]:
plot_pred(prediction_probabilities = prediction,
          lables = val_lables,
          images = val_images)

In [ ]:
plot_pred(prediction_probabilities = prediction,
          lables = val_lables,
          images = val_images,
          n=77)

In [ ]:
def plot_pred_conf(prediction_probabilities, labels, n=1):
    """
    Plots top 10 highest prediction confidences along with the truth label for sample n
    """
    pred_prob, true_label = prediction_probabilities[n], labels[n]
    pred_label = get_pred_lable(pred_prob)
    top_10_pred_index = pred_prob.argsort()[-10:][::-1]
    top_10_pred_value = pred_prob[top_10_pred_index]
    top_10_pred_labels = unique_breeds[top_10_pred_index]

    top_plot = plt.bar(np.arange(len(top_10_pred_labels)),
                       top_10_pred_value,
                       color="gray")

    plt.xticks(np.arange(len(top_10_pred_labels)),
               labels=top_10_pred_labels,
               rotation="vertical")

    if np.isin(true_label, top_10_pred_labels):
        top_plot[np.argmax(top_10_pred_labels == true_label)].set_color("green")


In [ ]:
plot_pred_conf(prediction_probabilities=prediction,
               labels = val_lables,
               n=9)

In [ ]:
i_multiplier = 0
num_cols = 2
num_rows = 3
num_images = num_cols*num_rows
plt.figure(figsize=(10*num_cols , 5*num_rows))
for i in range(num_images):
  plt.subplot(num_rows,2*num_cols, 2*i+1)
  plot_pred(prediction_probabilities=prediction,
            lables = val_lables,
            images=val_images,
            n = i+i_multiplier)
  plt.subplot(num_rows,2*num_cols, 2*i+2)
  plot_pred_conf(prediction_probabilities=prediction,
            labels = val_lables,
            n=i+i_multiplier)
plt.tight_layout(h_pad=1.0)
plt.show()

In [ ]:
!pip install tf-keras-vis

from tf_keras_vis.gradcam_plus_plus import GradcamPlusPlus
from tf_keras_vis.utils.scores import CategoricalScore
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear


In [ ]:
n=99
img = val_images[n]
img_batch = np.expand_dims(img, axis=0)
pred=model.predict(img_batch)
target_class_index = np.argmax(pred[0])

In [ ]:
import cv2
def show_gradcam(model,val_images,n):
  img = val_images[n]
  img_batch = np.expand_dims(img, axis=0)
  pred=model.predict(img_batch)
  target_class_index = np.argmax(pred[0])

  heatmap = GradcamPlusPlus(model, model_modifier=ReplaceToLinear(), clone=True)(
    CategoricalScore(target_class_index), img_batch)[0]

  # Normalize
  heatmap = np.maximum(heatmap, 0)
  heatmap = heatmap / (heatmap.max() + 1e-8)

  # Resize to match original
  heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

  # Threshold (makes output SUPER CLEAR)
  heatmap[heatmap < 0.7] = 0

  # Color map
  heatmap_color = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)

  # Soft overlay
  overlay = cv2.addWeighted(np.uint8(img * 255), 0.8, heatmap_color, 0.2, 0)

  plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.show()
  return target_class_index, pred[0]


In [ ]:
show_gradcam(model,val_images,n=59)

In [ ]:
def save_model(model, suffix="model"):
    """
    Saves a given model in a models directory and appends a suffix (string).
    """
    # Create directory path
    model_dir = os.path.join("drive/MyDrive/Dog Vision/models",
                             datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    os.makedirs(model_dir, exist_ok=True)  # ensure directory exists

    # Full path for the model file
    model_path = os.path.join(model_dir, f"{suffix}.h5")
    print(f"Saving model to {model_path}....")
    model.save(model_path)

    return model_path


In [ ]:
# create a model to lode a trained model
def load_model(model_path):
    print(f"Loading saved model from: {model_path}")
    return tf.keras.models.load_model(model_path)  



In [ ]:
# Save our model trained on 1000 images
save_model(model, suffix="1000-images-Adam")

In [ ]:
from tensorflow.keras.models import load_model
import tensorflow_hub as hub  

loaded_model = load_model(
    "drive/MyDrive/Dog Vision/models/20251013-055120/1000-images-Adam.h5",
    custom_objects={"KerasLayer": hub.KerasLayer} 
)



In [ ]:
model.save("model.h5")


In [ ]:
model.evaluate(valid_data)

In [ ]:
loaded_model.evaluate(valid_data)

In [ ]:
full_data = create_data_batches(X,y)
full_data

In [ ]:
full_model = create_model()

In [ ]:
full_model_tensorboard = create_tensorboard_callback()

# no validation setwhen traning all the data , so we can't monitar validation accuracy
full_model_early_stopping = tf.keras.callbacks.EarlyStopping(monitor="accuracy",
                                                             patience = 3)


In [ ]:
!pip install lime


In [ ]:
from lime import lime_image
from skimage.segmentation import mark_boundaries

explainer = lime_image.LimeImageExplainer()

explanation = explainer.explain_instance(
        img.astype('double'),
        model.predict,
        top_labels=5,
        hide_color=0,
        num_samples=1000)

temp, mask = explanation.get_image_and_mask(
        label=explanation.top_labels[0],
        positive_only=True,
        hide_rest=False)

plt.imshow(mark_boundaries(temp / 255.0, mask))
plt.show()

In [ ]:
# Fit the full model to the full data
full_model.fit(x=full_data,
               epochs=NUM_EPOCHS,
               callbacks=[full_model_tensorboard, full_model_early_stopping])

In [ ]:
save_model(full_model, suffix="full-image-set-mobilenetv2-Adam")

In [ ]:
# Load in the full model
loaded_full_model = load_model("drive/MyDrive/Dog Vision/models/20251013-065358/full-image-set-mobilenetv2-Adam.h5")

In [ ]:
# Load test image filenames
test_path = "drive/MyDrive/Dog Vision/test/test/"
test_filenames = [test_path + fname for fname in os.listdir(test_path)]
test_filenames[:10]

In [ ]:
len(test_filenames)

In [ ]:
import tensorflow as tf
import os

def create_data_batches(image_paths, batch_size=32, test_data=False):
    # Keep only files
    image_paths = [p for p in image_paths if os.path.isfile(p)]

    def preprocess(path):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [224, 224])
        img = img / 255.0
        return img

    dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


In [ ]:
test_dataset = create_data_batches(test_filenames, batch_size=64)



In [ ]:
test_predictions = loaded_full_model.predict(test_dataset, verbose=1)

In [ ]:
test_predictions[:10]

In [ ]:
# Create pandas DataFrame with empty columns
preds_df = pd.DataFrame(columns=["id"] + list(unique_breeds))
preds_df.head()

In [ ]:
test_path =  "drive/MyDrive/Dog Vision/test/test/"
preds_df["id"] = [os.path.splitext(path)[0] for path in os.listdir(test_path)]
preds_df.head()


In [ ]:
preds_df[list(unique_breeds)] = test_predictions
preds_df.head()

In [ ]:
# Save our predictions dataframe to CSV for submission to Kaggle
preds_df.to_csv("drive/MyDrive/Dog Vision/full_model_predictions_submission_1_mobilenetV2.csv",
                index=False)

In [ ]:
custom_path = "drive/MyDrive/Dog Vision/dogs/"
custom_image_paths = [custom_path + fname for fname in os.listdir(custom_path)]

In [ ]:
custom_image_paths

In [ ]:
custom_data = create_data_batches(custom_image_paths, test_data=True)
custom_data

In [ ]:
custom_preds = loaded_full_model.predict(custom_data)

In [ ]:
custom_preds.shape

In [ ]:
custom_pred_labels = [get_pred_lable(custom_preds[i]) for i in range(len(custom_preds))]
custom_pred_labels

In [ ]:
custom_images = [img for img in custom_data]


In [ ]:
import matplotlib.pyplot as plt
custom_images = np.array(custom_images)

num_to_show = min(4, len(custom_images))

plt.figure(figsize=(16, 4))
for i in range(num_to_show):
    plt.subplot(1, num_to_show, i+1)
    plt.axis('off')

    img = custom_images[i]

    if img.ndim == 4:
        img = img[0]
    if img.max() <= 1.0:
        img = (img * 255).astype(np.uint8)
    else:
        img = img.astype(np.uint8)

    plt.imshow(img)
    plt.title(custom_pred_labels[i] if i < len(custom_pred_labels) else "N/A")
plt.show()
